# Artificial Neural Network

Necessary library impor

In [73]:

import pandas as pd
import numpy as np
import pickle
import plotly.express as px, matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix

Load dataset

In [49]:
df = pd.read_csv("../dataset/Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [50]:
df.shape

(10000, 14)

In [51]:
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

drop unnecessary columns, split x & y

In [52]:
x = df.drop(columns=["RowNumber","Surname","Exited"], axis=1)
x.head()

,CustomerId,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,15634602,619,France,Female,42,2,0.00,1,1,1,101348.88
1,15647311,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,15619304,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,15701354,699,France,Female,39,1,0.00,2,0,0,93826.63
4,15737888,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [53]:
y = df["Exited"]
y

0       1
1       0
2       1
3       0
4       0
       ..
9995    0
9996    0
9997    1
9998    1
9999    0
Name: Exited, Length: 10000, dtype: int64

Encode categorical data

In [54]:
le = LabelEncoder()
x['Gender'] = le.fit_transform(x['Gender'])

One Hot encoding for Geography

In [55]:
oneh = OneHotEncoder(drop='first')
geo_en = oneh.fit_transform(x[['Geography']]).toarray()
geo_en_x = pd.DataFrame(geo_en, columns=oneh.get_feature_names_out(['Geography']))
geo_en_x

,Geography_Germany,Geography_Spain
0,0.0,0.0
1,0.0,1.0
2,0.0,0.0
3,0.0,0.0
4,0.0,1.0
...,...,...
9995,0.0,0.0
9996,0.0,0.0
9997,0.0,0.0
9998,1.0,0.0


In [56]:
x = pd.concat([x.drop(columns=['Geography']), geo_en_x], axis=1)
x.head()

,CustomerId,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain
0,15634602,619,0,42,2,0.00,1,1,1,101348.88,0.0,0.0
1,15647311,608,0,41,1,83807.86,1,0,1,112542.58,0.0,1.0
2,15619304,502,0,42,8,159660.80,3,1,0,113931.57,0.0,0.0
3,15701354,699,0,39,1,0.00,2,0,0,93826.63,0.0,0.0
4,15737888,850,0,43,2,125510.82,1,1,1,79084.10,0.0,1.0


Train test split

In [57]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

Feature scaling

In [58]:
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

# Build ANN Model

In [59]:
model = Sequential()

Input + Hidden Layers

In [60]:
model.add(Dense(64, activation='relu', input_shape=(x_train.shape[1],)))
model.add(Dropout(0.3))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(16, activation='relu'))
model.add(Dropout(0.4))

c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Output Layer

In [61]:
model.add(Dense(1, activation='sigmoid'))

In [62]:
# Compile
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics = ['accuracy']
)

In [63]:
#Train

es = EarlyStopping(
    monitor = 'val_loss',
    patience=10,
    restore_best_weights=True
)

model.fit(x_train, y_train, epochs=100, batch_size=32, validation_split=0.1, callbacks=[es])

Epoch 1/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7653 - loss: 0.5309 - val_accuracy: 0.8213 - val_loss: 0.4152
Epoch 2/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8061 - loss: 0.4627 - val_accuracy: 0.8512 - val_loss: 0.3891
Epoch 3/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8153 - loss: 0.4443 - val_accuracy: 0.8512 - val_loss: 0.3756
Epoch 4/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8260 - loss: 0.4226 - val_accuracy: 0.8562 - val_loss: 0.3677
Epoch 5/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8265 - loss: 0.4102 - val_accuracy: 0.8587 - val_loss: 0.3562
Epoch 6/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8378 - loss: 0.3956 - val_accuracy: 0.8600 - val_loss: 0.3449
Epoch 7/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8343 - loss: 0.3852 - val_accuracy: 0.8562 - val_loss: 0.3405
Epoch 8/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8421 - loss: 0.3901 - val_accu

In [64]:
# model evaluate
loss, accuracy = model.evaluate(x_test, y_test)
print("Test Accuracy:", accuracy)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8590 - loss: 0.3399 
Test Accuracy: 0.859000027179718


# Output Prediction Example

In [65]:
y_pred = model.predict(x_test)
y_pred = (y_pred > 0.5).astype(int)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


check classification report

In [66]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.96      0.92      1607
           1       0.73      0.45      0.56       393

    accuracy                           0.86      2000
   macro avg       0.80      0.70      0.74      2000
weighted avg       0.85      0.86      0.85      2000



chech confusion matrix

In [70]:
cm = (confusion_matrix(y_test, y_pred))

# Visualize Confusion Matrix

In [71]:
cm_df = pd.DataFrame(cm, index = ["Actual Negative", "Actual Positive"],
                     columns= ["Predicted Negative", "Predicted Positive"])
cm_df

,Predicted Negative,Predicted Positive
Actual Negative,1541,66
Actual Positive,216,177


In [72]:
fig = px.imshow(cm_df,
                text_auto = True,
                color_continuous_scale= 'Viridis',
                title= "Confusion Matrix")
fig.update_layout(
    title={
        'text':'Confusion Matrix',
        'x': 0.5,
    },
    xaxis_title= "Prediction Label",
    yaxis_title= "True Label",
    coloraxis_showscale= True
)
fig.show()

# Save Model

save standard scaling


In [76]:
with open("../model/standard_scaler.pkl", "wb") as f:
    pickle.dump(sc, f)

save ann model

In [78]:
model.save("../model/churn_ann_model.h5")